# UE2 - Feature Engineering und Klassifikation mit sklearn
Author: Bleyel Andreas

## Aufgabe 2d Kaggle Challenge
https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, median_absolute_error
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV

pd.set_option('display.max_columns', 100)

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

### Inspektion des Datensets

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
train.head(3)

Zu Beginn ist gleich einmal zu sehen, dass wir es mit sehr vielen Feature-Variablen zu tun haben. Eine Reduktion auf solche welche einen signifikanten Einfluss auf die Vorhersage der Ziel-Variable "Sales Price" haben, hat die höchste Priorität und wird zuerst behandelt.

In [ ]:
corr = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr, square=True);

TotalBsmtSF mit 1FlrSF sowie GarageCars und GarageArea scheine eine hohe Korrelation zu haben. Dies macht auch Sinn da 1stFlrSF und TotalBsmtSF insofern korrelieren, da die Gesamtanzahl der m² und die des ersten Stockes meist sehr ähnlich wenn nicht sogar gleich sind. Genauso ist es nur logisch und leicht erklärbar, dass umso mehr Platz für Autos in einer Garage sind, auch die Fläche der Garage steigt. Zu hinterfragen wäre lediglich, ob eine Multikolinearität vorliegt und es nicht besser wäre, eine der beiden Variablen zu entfernen. Dazu dann später mehr wenn fehlende Werte untersucht werden und sich vielleicht ohnehin eine Variable dadurch von selbst anbietet.

Weitere Pärchen welche korrelieren:

* OverallQual scheint auch mit vielen anderen Variablen zu korrelieren. Auch das erscheint plausibel da schon die Bezeichnung "Overall" auf eine Bewertung von mehreren Attributen hindeutet. Vor allem mit SalePrice scheint (wenig überraschend) eine hohe Korrelation vorzuliegen.
* YearBuilt gemeinsam mit GaragyYrBlt ist auch rational erklärbar.
* GrLivArea und TotRmsAbvGrd.

Variablen welche mit SalesPrice in deutlichem Zusammenhang stehen und einer nähere Betrachtung Wert sind:

* OverallQual
* GarageArea / Cars
* GrLiveArea
* TotalBsmtSF

Der nächste Schritt ist es zu prüfen, welche Variable des Pärchens GarageArea/GarageCars wir für unser Modell genommen wird. GarageArea gibt vor, wieviele Cars Platz finden und bietet sich somit für eine Beibehaltung an. Trotzdem noch ein paar Checks um sicher zu gehen.

In [ ]:
checkFeature('GarageArea')

In [ ]:
checkFeature('GarageCars')

Soweit keine fehlenden Werte aber natürlich Datensätze welche den Wert 0 in beiden Spalten enthalten. Dies sind Anwesen ohne Garage was soweit Sinn ergibt. Auch die Anzahl ist ident. Daher belassen wir es vorerst bei GarageArea für die weitere Bearbeitung.

Bevor wir unter die Haube der Ziel-Variable SalesPrice blicken, möchte Ich noch einmal die Variable OverallQual etwas näher betrachten. 

In [ ]:
checkFeature('OverallQual')

OverallQual ist eine Aufteilung in 10 Wertungs-Kategorien. Beginnend bei 1 als schlechteste Kategorie, bishin zu 10 als beste. Dabei handelt es sich um int64 Ganzzahlen welche aber besser als kategoriale Variable dargestellt werden sollten.

In [ ]:
k = 10 
cols = corr.nlargest(k, 'OverallQual')['OverallQual'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Im Vergleich dazu, die 10 Variablen welche den größten Einfluss auf SalePrice haben: 

In [ ]:
cols = corr.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Man sieht schon, dass einige Variablen in beiden Diagrammen vorkommen. Diese werden in späterer Folge wohl eine wichtige Rolle spielen, daher nachfolgend eine Auflistung mit kurzer Beschreibung:

* YearBuilt - wann das Objekt erbaut wurde
* GarageAreas - Platz der Garage in square feet
* GrLivingArea - Wohnbereich über dem Grund in square fett
* FullBath - Volle Badzimmer über dem Grund. Interessat wäre noch zu wissen, was mit "Voll" gemeint ist.
* TotalBsmtSF - Summe der square feet des Untergeschosses
* OverallQual - Gesamtzustand des Objekts

## Analyse der Feature-Variablen
Kommen wir zurück zu unseren Feature-Variablen. Diese werden wir in den nächsten Schritten einzeln unter die Lupe nehmen und gegebenfalls anpassungen vornehmen.

### YearBuilt
Wann das Objekt erbaut wurde. Erwartet werden Ganzzahlen im üblichen Jahresformat, keine negativen und keine fehlenden Werte.

In [ ]:
checkFeature('YearBuilt')

Es war zu erwarten, dass wir hier keine Normalverteilung vorfinden werden. Derzeit ist die Variable noch als numersicher Wert im Datenset. Eine Umwandlung durch OneHotEnconding ist je nach eingesetztem Algorithmus abzuwägen. Auch eine Einteilung in Kategorien wie zB "vor 1900", "1900-1930", "vor 1.WK" etc. könnte eine nähere Untersuchung wert sein. Vorerst ist wichtig, dass wir keine negativen oder fehlenden Werte haben. 

In Verbindung mit SalePrice ist eine schwache logarithmische Kurve (Anstieg) zu erkennen. Interessant wäre zu wissen, ob die Verkaufspreise der Vergangenheit Inflationsbereinigt wurden.

### GarageArea
Platz der Garage in square feet. Erwartet werden float64 Werte, keine negativen und keine fehlenden Werte.

In [ ]:
checkFeature('GarageArea')

In [ ]:
printSkewKurt('GarageArea')

Die Verteilung geht schon in Richtung Normalverteilung könnte aber vielleicht durch eine log-Transformierung verbessert werden. Dies prüfen wir im nächsten Schritt. Natürlich gibt es die "Ausreißer" mit 0ft² von Objekten welche keine Garage besitzen. Um diese müssen wir uns zuvor kümmer da wir 0 Werte nicht log-Transformieren können. 

Auffallend die die 4 Ausreißer rechts unten welche auf sehr große Garagen aber trotzdem einen niedrigen Verkaufspreis darstellen. Werfen wir einen genaueren Blick auf diese 4 Objekte.

In [ ]:
train.nlargest(4, columns=['GarageArea'])

Bei diesen 4 Objekten konnten keine Auffälligkeiten gefunden werden. Belassen wir sie im Datensatz. Die Beziehung zwischen GarageArea und SalePrice ist wenn dann nur schwach linear. 

Um die log-Transformierung durchzuführen fügen wir dem Datenset die Spalte "hasGarage" hinzu welchen einen Binärwert halten wird. 0=keine Garage 1=hat Garage

In [ ]:
train['HasGarage'] = pd.Series(len(train['GarageArea']), index=train.index)
train['HasGarage'] = 0 
train.loc[train['GarageArea']>0,'HasGarage'] = 1

Nun können wir die log-Transformierung nur auf die Zeilen anwenden, welche in "hasGarage" 1 haben. 

In [ ]:
train.loc[train['HasGarage']==1,'GarageArea'] = np.log(train['GarageArea'])

In [ ]:
sns.distplot(train[train['GarageArea']>0]['GarageArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(train[train['GarageArea']>0]['GarageArea'], plot=plt)

In [ ]:
print("Skewness: %f" % train[train['GarageArea']>0]['GarageArea'].skew())
print("Kurtosis: %f" % train[train['GarageArea']>0]['GarageArea'].kurt())

Es scheint als hätten wir uns ein wenig von der Normalverteilung wegbewegt weshalb wird von einer log-Transformierung von "GarageArea" absehen werden.

### GrLivArea
Wohnbereich über dem Grund in square feet. Erwartet werden float64 Werte, keine negativen und keine fehlenden Werte.

In [ ]:
checkFeature('GrLivArea')

In [ ]:
printSkewKurt('GrLivArea')

Verteilung scheint in Ordnung und es liegt eine lineare Beziehung vor. Dies war auch so zu erwarten dass mit steigender Größe des Wohnbereichs auch der Verkaufspreis ansteigt.

Trotzdem starten wir auch hier den Versuch einer log-Transformation und sehen wie sich die Verteilung ändert.

In [ ]:
train['GrLivArea'] = np.log(train['GrLivArea'])

In [ ]:
checkFeature('GrLivArea')

In [ ]:
printSkewKurt('GrLivArea')

Sieht deutlich besser aus weshalb wir die log-Transformierung beibehalten werden.

### FullBath

Volle Badzimmer über dem Grund. Erwartet werden Ganzzahlen, keine negativen und keine fehlenden Werte.

In [ ]:
checkFeature('FullBath')

Wir sehen, es gibt 4 Kategorien von Bädern. 0-4 Stück pro Objekt. Eine Umwandlung in kategorische Variablen ist nicht wünschenswert da wir die Wertung beibehalten wollen. Mehr Badezimmer sind nunmal mehr wert als weniger.

### TotalBsmtSF
Summe der square feet des Kellers. Erwartet werden float Werte, keine negativen und keine fehlenden Werte.

In [ ]:
checkFeature('TotalBsmtSF')

Auch hier sieht es nach einer linearen Beziehung aus bei der eine schöne Verteilung vorliegt. Die einzige Sache welche zu bedenken ist, wie gehen wir mit den 0 Werten (kein Keller) um. Wir wenden die selbe Vorgehensweise als bei "GarageArea" an da die Ausgangssituation nahezu ident ist und prüfen, ob eine log-Transformation Sinn macht.

In [ ]:
printSkewKurt('TotalBsmtSF')

In [ ]:
train['HasBasement'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBasement'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBasement'] = 1

In [ ]:
train.loc[train['HasBasement']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])

In [ ]:
sns.distplot(train[train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(train[train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [ ]:
print("Skewness: %f" % train[train['TotalBsmtSF']>0]['TotalBsmtSF'].skew())
print("Kurtosis: %f" % train[train['TotalBsmtSF']>0]['TotalBsmtSF'].kurt())

Auch hier sieht es nach einer Verbesserung im Sinne von einer Annäherung an eine Normalverteilung aus weshalb die log-Transformierung beibehalten wird.

### OverallQual
Gesamtzustand des Objekts. 10 Kategorien von 1-10 wobei 1 die schlechteste und 10 die beste Bewertung darstellt.

In [ ]:
checkFeature('OverallQual')

### Zusammenfassung der Analyse der Feature-Variablen
Nach Begutachtung der vielversprechendsten Feature-Variablen wird das Trainings-Datenset wie folgt aufgebaut:

* YearBuilt - wird nicht geändert
* GarageAreas - wird nicht geändert (GarageCars wird weggelassen)
* GrLivingArea - log-Transformierung
* FullBath - wird nicht geändert
* TotalBsmtSF - log-Transformierung
* OverallQual - wird nicht geändert

## Ziel-Variable SalesPrice
Als nächste wird unsere Ziel-Variable etwas genauer unter die Lupe genommen. Die Erwartung ist, dass es sich um einen numerischen Wert handelt welcher angibt, um welchen USD Betrag die Immobilie verkauft wurde. Da es sich um einen Verkaufspreis handelt, nehme ich im Vorfeld an, dass es sich um keine Normalverteilung handeln wird und wir eine log-Transformierung vornehmen werden müssen. Negative Werte werden ebenfalls nicht erwartet. Lassen wir die Zahlen sprechen.

In [ ]:
checkFeature('SalePrice')

Wie erwartet bestätigen sich die beiden Annahmen, dass es sich um keine Normalverteilung handelt und keine negativen Werte vorhanden sind. Mittels Scatterplot-Matrix betrachten wir die Zusammenhänge unserer Ziel-Variable mit den bereits weiter oben gefundenen vielversprechenden Feature-Variablen
* YearBuilt - wann das Objekt erbaut wurde
* GarageArea - Platz der Garage in square feet
* GrLivingArea - Wohnbereich über dem Grund in square fett
* FullBath - Volle Badzimmer über dem Grund. Interessat wäre noch zu wissen, was mit "Voll" gemeint ist.
* TotalBsmtSF - Summe der square feet des Keller

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'GarageArea']
sns.pairplot(train[cols], size = 2.5)
plt.show();

Die Scatter-Plots zeigen keine Überraschungen. SalesPrice zeigt 4 Punkte welche herausstechen und möglicherweise Outlier sind. Bei TotalBsmtSF ist vor allem ein Punkt sehr markant weit außen und einer näheren Betrachtung wert. Wir wollen aber zuerst noch SalesPrice weiter analysieren und beginnen mit deren Skalierung und log-Transformation.

In [ ]:
y = train['SalePrice'].values.reshape(-1,1)

In [ ]:
robust_scaler_y = RobustScaler()
robust_scaler_y.fit(y)
salePrice_scaled_robust = robust_scaler_y.transform(y)

In [ ]:
f, (ax0, ax1) = plt.subplots(1, 2)

ax0.hist(train['SalePrice'].values, bins=100)
ax0.set_ylabel('Price')
ax0.set_xlabel('Target')
ax0.set_title('SalePrice distribution')

ax1.hist(salePrice_scaled_robust, bins=100)
ax1.set_ylabel('Price')
ax1.set_xlabel('Target')
ax1.set_title('SalePrice-scaled distribution')

f.suptitle("Vergleich unscaled und RobustScaler", y=0.035)
f.tight_layout(rect=[0.05, 0.05, 0.95, 0.95])

Zeit die log-Transformation anzuwenden.

In [ ]:
train['SalePrice'] = np.log(train['SalePrice'])
checkFeature('SalePrice')

In [ ]:
train.head(3)

Nach Skalierung und der log-Transformation ist jetzt eine besser Verteilung der Ziel-Variable gegeben. Der Probability-Plot (eine Spezialform des Q-Q Plots) zeigt uns, dass die Daten nahe an der roten Linie liegen was auf eine Normalverteilung deutet. Dies wird durch das darüber befindliche Histogramm ebenfalls bestätigt wodurch unsere Ziel-Variable soweit für die Modellierung vorbereitet ist.

### Aufbau Test und Train Datenset

In [3]:
features = ['YearBuilt','GarageArea','OverallQual', 'GrLivArea', 'TotalBsmtSF', 'FullBath']

In [4]:
test_reduced = pd.read_csv('test.csv')[features].copy()

In [5]:
test_reduced.fillna(test_reduced.mean(), inplace=True)

In [6]:
test_reduced.isna().sum()

YearBuilt      0
GarageArea     0
OverallQual    0
GrLivArea      0
TotalBsmtSF    0
FullBath       0
dtype: int64

In [7]:
test_reduced['HasBasement'] = pd.Series(len(test_reduced['TotalBsmtSF']), index=test_reduced.index)
test_reduced['HasBasement'] = 0
test_reduced.loc[test_reduced['TotalBsmtSF']>0,'HasBasement'] = 1
test_reduced.loc[test_reduced['HasBasement']==1,'TotalBsmtSF'] = np.log(test_reduced['TotalBsmtSF'])
test_reduced['GrLivArea'] = np.log(test_reduced['GrLivArea'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [8]:
test_reduced.drop('HasBasement', inplace=True, axis=1)

In [9]:
test_reduced.head(3)

,YearBuilt,GarageArea,OverallQual,GrLivArea,TotalBsmtSF,FullBath
0,1961,730.0,5,6.797940,6.782192,1
1,1958,312.0,6,7.192182,7.192182,1
2,1997,482.0,5,7.395722,6.833032,2


In [10]:
test_reduced.shape

(1459, 6)

In [11]:
features.append('SalePrice')
features

['YearBuilt',
 'GarageArea',
 'OverallQual',
 'GrLivArea',
 'TotalBsmtSF',
 'FullBath',
 'SalePrice']

In [12]:
train_reduced = pd.read_csv('train.csv')[features].copy()

In [13]:
train_reduced['HasBasement'] = pd.Series(len(train_reduced['TotalBsmtSF']), index=train_reduced.index)
train_reduced['HasBasement'] = 0 
train_reduced.loc[train_reduced['TotalBsmtSF']>0,'HasBasement'] = 1
train_reduced.loc[train_reduced['HasBasement']==1,'TotalBsmtSF'] = np.log(train_reduced['TotalBsmtSF'])
train_reduced['GrLivArea'] = np.log(train_reduced['GrLivArea'])
train_reduced['SalePrice'] = np.log(train_reduced['SalePrice'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [15]:
train_reduced.drop('HasBasement', inplace=True, axis=1)

In [16]:
train_reduced.head(3)

,YearBuilt,GarageArea,OverallQual,GrLivArea,TotalBsmtSF,FullBath,SalePrice
0,2003,548,7,7.444249,6.752270,2,12.247694
1,1976,460,6,7.140453,7.140453,2,12.109011
2,2001,608,7,7.487734,6.824374,2,12.317167


In [17]:
train_reduced.shape

(1460, 7)

In [18]:
train_reduced.isna().sum()

YearBuilt      0
GarageArea     0
OverallQual    0
GrLivArea      0
TotalBsmtSF    0
FullBath       0
SalePrice      0
dtype: int64

## Prädiktion

In [19]:
features.remove('SalePrice')

In [20]:
y = train_reduced['SalePrice']
X = train_reduced[features]

### Lineares Modell

In [21]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.score(X, y)

0.8163983013880629

### Random Forrest

Einfach ohne Parameter Tuning:

In [22]:
forest_regressor = RandomForestRegressor(n_estimators=10)
forest_regressor.fit(X, y.ravel())
forest_regressor.score(X, y)

0.9683352873474684

In [23]:
forest_pred_y = forest_regressor.predict(test_reduced)
forest_pred_y = np.exp(forest_pred_y)
forest_pred_y

array([131039.47827211, 139129.46229033, 150250.09426707, ...,
       151697.64245361, 100187.96538208, 231177.9386026 ])

Hyperparameter Tuning mittels K-Fold und Gridsearch

In [32]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [34]:
rf_tuned = RandomForestRegressor()

# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_tuned, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X, y.ravel())

## WICHTIG: Laufzeit mit 5Kfold ca 10min

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  9.4min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [35]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [ ]:
rf_tuned.fit(X, y.ravel())
rf_tuned.score(X, y)

In [28]:
rf_tuned_pred_y = rf_tuned.predict(test_reduced)
rf_tuned_pred_y = np.exp(rf_tuned_pred_y)
rf_tuned_pred_y

array([129775.90385113, 148534.08092256, 154451.92017118, ...,
       144064.0545659 , 111011.43176321, 232733.85103649])

### Submission File erzeugen

In [30]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': rf_tuned_pred_y})
my_submission.to_csv('submission.csv', index=False)

## Helper Funktionen

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
target = 'SalePrice'

def checkFeature(feature):
    checkNAs(feature)
    checkForNegatives(feature)
    overview(feature)
    plotDistribution(feature)
    if feature != target:
        plotRelationToTarget(feature)
    
def checkNAs(feature):
    if train[feature].isna().sum() > 0:
        print(bcolors.FAIL + "Sum NAs: " + str(train[feature].isna().sum()))
    else:
        print(bcolors.OKGREEN + "No NAs" +bcolors.ENDC)

def checkForNegatives(feature):
    if any(train[feature]<0):
        print (bcolors.WARNING + "Warning feature has negative value!" + bcolors.ENDC)
    else:
        print (bcolors.OKGREEN + "No negative values" + bcolors.ENDC)

def plotDistribution(feature):
    sns.distplot(train[feature], fit=norm);
    fig = plt.figure()
    res = stats.probplot(train[feature], plot=plt)
    
def plotRelationToTarget(feature):
    data_temp = pd.concat([train[target], train[feature]], axis=1)
    data_temp.plot.scatter(x=feature, y=target, ylim=(0,800000));
        
def overview(feature):
    print(train[feature].describe())
    print(bcolors.HEADER + "Head" +bcolors.ENDC)
    print(train[feature].head(3))
    
def printSkewKurt(feature):
    print("Skewness: %f" % train[feature].skew())
    print("Kurtosis: %f" % train[feature].kurt())
    
def calculate_performance(prediction, actual, scaler):
    if scaler == True:
        p = scaler.inverse_transform(prediction.reshape(-1,1))
        a = scaler.inverse_transform(actual.reshape(-1,1))
    else:
        p = prediction
        a = actual
        
    mse = mean_squared_error(a, p)
    err = np.sqrt(mse)
    r2 = r2_score(a, p)
    mae = median_absolute_error(a, p)
    
    return (mse, err, r2, mae)

def print_performance(measure_tuple):
    
    mse = measure_tuple[0]
    err = measure_tuple[1]
    r2 = measure_tuple[2]
    mae = measure_tuple[3]
    
    print("Mean squared error is {}".format(str(mse)))
    print("Positive mean error is {}".format(str(err)))
    print("Overall R² is {}".format(str(r2)))
    print("Median absolute error is {}".format(str(mae)))